# Detect Heart Failure from Clinical Record with SageMaker FeatureStore
This notebook runs with Kernel `Python 3 (Data Science)`.

Note:

The following policies need to be attached to the execution role that you used to run this notebook:

* AmazonSageMakerFullAccess
* AmazonSageMakerFeatureStoreAccess
* AmazonS3FullAccess

Note that the `AmazonS3FullAccess` policy is not attached to your role by default if you choose to `create a new role` when you start your SageMaker Studio instance. If you don't see the required policies above are listed under `Policy name`, you can go to the IAM console, find your role, choose `Attach Policies` under `Permissions`, find the policies you are missing from the list, then choose `Attach policy`. 

## Contents
* [Background](#1)
* [Architecture](#10)
* [Setup SageMaker FeatureStore](#2)
* [Inspect Dataset](#3)
* [Prepare Data for FeatureStore](#4)
* [Create Features](#5)
* [Work with FeatureGroup](#10)
* [Build Training Dataset](#6)
* [Train and Deploy the Model](#7)
* [SageMaker FeatureStore At Inference](#8)
* [Cleanup Resources](#9)

<a id='1'></a>
## Background

SageMaker FeatureStore is a new SageMaker capability that makes it easy for customers to create and manage curated features for machine learning (ML) development. It serves as the single source of truth to store, retrieve, remove, track, share, discover, and control access to features.
SageMaker FeatureStore enables data ingestion via a high TPS API and data consumption via the online and offline stores.


This notebook provides an example for the APIs provided by SageMaker FeatureStore by walking through the process of training a heart failure detection model with clinical records data. The notebook demonstrates how the dataset can be ingested into the FeatureStore, queried to create a training dataset, and quickly accessed during inference.

### Terminology
* `Feature group` – A FeatureGroup is the main Feature Store resource that contains the metadata for all the data stored in Amazon SageMaker Feature Store. A feature group is a logical grouping of features, defined in the feature store, to describe records. A feature group’s definition is composed of a list of feature definitions, a record identifier name, and configurations for its online and offline store. 

* `Feature definition` – A FeatureDefinition consists of a name and one of the following data types: an Integral, String or Fractional. A FeatureGroup contains a list of feature definitions. 

* `Record identifier name` – Each feature group is defined with a record identifier name. The record identifier name must refer to one of the names of a feature defined in the feature group's feature definitions. 

* `Event time` – a point in time when a new event occurs that corresponds to the creation or update of a record in a feature group. All records in the feature group must have a corresponding Eventtime. It can be used to track changes to a record over time. The online store contains the record corresponding to the last Eventtime for a record identifier name, whereas the offline store contains all historic records.

* `Online Store` – the low latency, high availability cache for a feature group that enables real-time lookup of records. The online store allows quick access to the latest value for a Record via the GetRecord API. A feature group contains an OnlineStoreConfig controlling where the data is stored.

* `Offline store` – the OfflineStore, stores historical data in your S3 bucket. It is used when low (sub-second) latency reads are not needed. For example, when you want to store and serve features for exploration, model training, and batch inference. A feature group contains an OfflineStoreConfig controlling where the data is stored.

<a id='10'></a>
## Solution Architecture Diagram
----

![Solution Archtecture](images/fs1.png)


<a id='2'></a>
## Setup SageMaker FeatureStore
Let's start by setting up the SageMaker Python SDK and boto client. 

In [ ]:
!pip install s3fs

In [ ]:
import boto3
import sagemaker
from sagemaker.session import Session


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

#### Set Up S3 Bucket For The OfflineStore
SageMaker FeatureStore writes the data in the `OfflineStore` of a `FeatureGroup` to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you. Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

In [ ]:
# change the bucket name to your desired bucket name 
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'feature-store-demo'

print(default_s3_bucket_name)

#### Set up IAM Role

In [ ]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)

<a id='3'></a>

## Inspect Dataset

The [Heart Failure Clinical Dataset](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records) contains electronic medical records of patients quantify symptoms, body features, and clinical laboratory test values of 299 patients with heart failure in 2015.

The dataset contains one table with thirteen (13) columns:

- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days). To clarify, the time column here is not the event time column we just mentioned, but is the days between the last time the patient was seen and the time of the follow-up happens to check if the patient has had a heart failure. We will create the event time column later in this demo.
- (target)death event: if the patient deceased during the follow-up period (boolean)

The objective of the model is to predict patients’ survival from their clinical records data.


In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
#download data from online source
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv -O data/clinical_records_dataset.csv

In [ ]:
!aws s3 cp  ./data/clinical_records_dataset.csv s3://$default_s3_bucket_name/$prefix/data/ 

In [ ]:
clinical_data_file_name = 'clinical_records_dataset.csv'
clinical_data_path = "s3://{}/{}/data/{}".format(default_s3_bucket_name, prefix, clinical_data_file_name)
clinical = pd.read_csv(clinical_data_path)
pd.set_option('display.max_columns', 500)
clinical.head()

In [ ]:
print ('percentage of the value missing in each column is: ')
clinical.isnull().sum() / len(clinical)

The dataset contains no missing value, and all columns are either numerical or binary, therefore no processing or feature engineering is needed in this case. Depending on your data and use case, you should examine your data and decide if any pre-processing and feature engineering steps are needed before you ingest your data into FeatureStore.

<a id='4'></a>

## Prepare data For FeatureStore
In the Amazon SageMaker Feature Store API, a feature is an attribute of a record. You can define a name and type for every feature stored in Feature Store. Name uniquely identifies a feature within a feature group. Type identifies the datatype for the values of the feature. Supported data types are: String, Integral and Fractional. 

Take a look at the data types and making sure they are all correct and readable by Feature store. SageMaker FeatureStore Python SDK will map the string dtype to String feature type.

In SageMaker Feature Store, a `record` is a collection of values for features for a single record identifier value. Specific features are flagged with record identifier and event time, and a combination of record identifier name and a timestamp uniquely identify a record within a feature group. we will need to specify a record identifier and an event time in this case, and since the raw data does not contain the two columns, we will need to create them.

* For record identifier name: a record is a collection of values for features for a single record identifier value. In this case, we will create a unique ID for each patient in the previous step as the record identifier. Making sure the identifier is the unique identifier for each instance.
* For event time feature name: it refers to a point in time when a new event occurs that corresponds to the creation or update of a record in a feature group. It can be used to track changes to a record over time. For example, in this use case, EventTime can be appended to your data when no timestamp is available. In the following code, you can see how EventTime is appended to the clinical data.

#### Create a unique ID for each patient

In [ ]:
#### Add an id for each patient
clinical.reset_index(inplace = True)
clinical.rename(columns = {'index': 'patient_id'}, inplace = True)

In [ ]:
clinical.dtypes

In [ ]:
#### We want this id to be treated as a sting ID
clinical['patient_id'] = clinical['patient_id'].astype(object)

#### Create a TimeStamp for each Record

In [ ]:
import time

current_time_sec = int(round(time.time()))
# append EventTime feature
clinical['EventTime'] = pd.Series([current_time_sec]*len(clinical), dtype="float64")

#### Check data types for each column

In [ ]:
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(clinical)

In [ ]:
clinical.dtypes

<a id='5'></a>
## Create Features
In this step we will create the FeatureGroup representing the patients' clinical records, then ingest the data into the created FeatureGroup.

#### Assign a feature group name

In [ ]:
from time import gmtime, strftime, sleep

clinical_feature_group_name = 'clinical-feature-group-' + strftime('%d-%H-%M-%S', gmtime())

#### Create a FeatureGroup

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup

clinical_feature_group = FeatureGroup(name=clinical_feature_group_name, sagemaker_session=feature_store_session)

#### Define Identifier
 In this step, we will specify a record identifier name and an event time feature name. 

In [ ]:
# record identifier and event time feature names
record_identifier_feature_name = "patient_id"
event_time_feature_name = "EventTime"

#### Load feature definitions to the feature group
We can now load the feature definitions by passing a data frame containing the feature data. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data. For developers using a schema rather than automatic detection, see the [Export Feature Groups from Data Wrangler example](https://docs.aws.amazon.com/sagemaker/latest/dg/data-wrangler-data-export.html#data-wrangler-data-export-feature-store) for code that shows how to load the schema, map it, and add it as a FeatureDefinition that you can use to create the FeatureGroup. 

In [ ]:
clinical_feature_group.load_feature_definitions(data_frame=clinical); # output is suppressed

#### Create FeatureGroup
In this step, we will use the create function to create the feature group.  Note that the online store is not created by default, so you must set this as `True` if you want to enable it. The `s3_uri` is the S3 bucket location of your offline store. Check the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-create-feature-group.html) for a list of other parameters you can define.

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

clinical_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}", #offline feature store bucket
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True
)
wait_for_feature_group_creation_complete(feature_group=clinical_feature_group)


## Work with your FeatureGroup
#### Check FeatureGroup Info
When you create a feature group, it takes time to load the data, and you need to wait until the feature group is created before you can use it. You can check status using the DescribeFeatureGroup and ListFeatureGroups APIs.

In [ ]:
clinical_feature_group.describe()

In [ ]:
sagemaker_client.list_feature_groups() # use boto client to list FeatureGroups

#### Put Records into the Feature Store 
After the FeatureGroups have been created, we can put data into the FeatureGroups by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion. You can use the ingest function to load your feature data. You pass in a data frame of feature data, set the number of workers, and choose to wait for it to return or not. For this example, to accelerate the ingestion process, we are specifying multiple workers to do the job simultaneously. It will take <1min to ingest data to the Clinical FeatureGroup we created.

In [ ]:
clinical_feature_group.ingest(
    data_frame=clinical, max_workers=3, wait=True
)

#### Get Records from a Feature Group
We can use the get_record function to retrieve the data for a specific feature by its record identifier from the online store.

In [ ]:
record_identifier_value = str(200)

featurestore_runtime.get_record(FeatureGroupName=clinical_feature_group_name, RecordIdentifierValueAsString=record_identifier_value)


#### Generate Hive DDL Commands
The SageMaker Python SDK’s FeatureStore class also provides the functionality to generate Hive DDL commands. The schema of the table is generated based on the feature definitions. Columns are named after feature name and data-type are inferred based on feature type.

In [ ]:
print(clinical_feature_group.as_hive_ddl())

Now let's wait for the data to appear in our offline store before moving forward to creating a dataset. This will take approximately 2 minutes. SageMaker FeatureStore adds metadata for each record that's ingested into the offline store.

%%time
s3_client = boto3.client('s3', region_name=region)

account_id = boto3.client('sts').get_caller_identity()["Account"]
print(account_id)

clinical_feature_group_s3_prefix = prefix + '/' + account_id + '/sagemaker/' + region + '/offline-store/' + clinical_feature_group_name + '/data'

offline_store_contents = None
while (offline_store_contents is None):
    objects_in_bucket = s3_client.list_objects(Bucket=default_s3_bucket_name, Prefix=clinical_feature_group_s3_prefix)
    if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) >= 1):
        offline_store_contents = objects_in_bucket['Contents']
    else:
        print('Waiting for data in offline store...\n')
        sleep(60)
    
print('Data available.')

In [ ]:
%%time
s3_client = boto3.client('s3', region_name=region)
account_id = boto3.client('sts').get_caller_identity()["Account"]
clinical_feature_group_table_name = clinical_feature_group.describe().get('OfflineStoreConfig').get('DataCatalogConfig').get('TableName')
print(account_id)
print(clinical_feature_group_table_name)
clinical_feature_group_s3_prefix = prefix + '/' + account_id + '/sagemaker/' + region + '/offline-store/' + clinical_feature_group_table_name + '/data'
offline_store_contents = None
while (offline_store_contents is None):
    objects_in_bucket = s3_client.list_objects(Bucket=default_s3_bucket_name, Prefix=clinical_feature_group_s3_prefix)
    if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) >= 1):
        offline_store_contents = objects_in_bucket['Contents']
    else:
        print('Waiting for data in offline store...\n')
        sleep(60)
print('Data available.')

## Build a Training Dataset
  SageMaker FeatureStore automatically builds an AWS Glue data catalog when you create feature groups, and you can turn this off if you want. In this example, we will create a training dataset with FeatureValues from the clinical FeatureGroup. This is done by utilizing the auto-built Catalog. We run an Athena query that does a simple `select all` in the offline store in S3 from the FeatureGroup.
  
For testing purpose, we left out 9 records when creating the training dataset, so that we can use the left-out 9 records as test data for the reference. You can also do a train/test split. 

In [ ]:
clinical_query = clinical_feature_group.athena_query()
clinical_table = clinical_query.table_name

In [ ]:
# Athena query
query_string = 'SELECT * FROM "'+clinical_table+'" LIMIT 290'

# run Athena query. The output is loaded to a Pandas dataframe.
dataset = pd.DataFrame()
clinical_query.run(query_string=query_string, output_location='s3://'+default_s3_bucket_name+'/query_results/')
clinical_query.wait()
dataset = clinical_query.as_dataframe()

In [ ]:
id_for_test = []
for i in range(299):
    if i not in dataset['patient_id'].unique():
        id_for_test.append(i)

#### Prepare dataset for training

In [ ]:
# Prepare query results for training.
query_execution = clinical_query.get_query_execution()
query_result = 's3://'+default_s3_bucket_name+'/'+prefix+'/query_results/'+query_execution['QueryExecution']['QueryExecutionId']+'.csv'
print(query_result)

In [ ]:
# Select useful columns for training with target column as the first.
dataset = dataset[["death_event", "age", 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']]
# Write to csv in S3 without headers and index column.
dataset.to_csv('dataset.csv', header=False, index=False)
s3_client.upload_file('dataset.csv', default_s3_bucket_name, prefix+'/training_input/dataset.csv')
dataset_uri_prefix = 's3://'+default_s3_bucket_name+'/'+prefix+'/training_input/';

In [ ]:
dataset.head(2)

## Train and Deploy the Model
For model training, we will use a SageMaker built-in algorithm called XGBoost to predict if a patient is likely to have a heart failure. SageMaker built-in algorithms provide highly optimized implementation of popular machine learning algorithms, simplifying the machine learning development and accelerating training and deployment. We will call the SageMaker XGBoost container and construct a generic SageMaker estimator.

In [ ]:
training_image=sagemaker.image_uris.retrieve("xgboost", region, "1.0-1")

In [ ]:
training_output_path='s3://' + default_s3_bucket_name+'/'+prefix + '/training_output'

from sagemaker.estimator import Estimator
training_model = Estimator(training_image,
                           role, 
                           instance_count=1, 
                           instance_type='ml.m5.2xlarge',
                           volume_size = 5,
                           max_run = 3600,
                           input_mode= 'File',
                           output_path=training_output_path,
                           sagemaker_session=feature_store_session)

Due to cost consideration, the goal of this example is to showcase FeatureStore capabilities, not necessarily to achieve the best result. In this example, we will skip hyperparamter tuning and go with default hyperparameters.

In [ ]:
training_model.set_hyperparameters(objective = "binary:logistic",
                                   num_round = 50)

#### Specify training dataset to the dataset we just created

In [ ]:
train_data = sagemaker.inputs.TrainingInput(dataset_uri_prefix, distribution='FullyReplicated', 
                                            content_type='text/csv', s3_data_type='S3Prefix')
data_channels = {'train': train_data}

In [ ]:
training_model.fit(inputs=data_channels, logs=True)

#### Set up Hosting for the Model
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. The endpoint deployment can be accomplished as follows. This takes 8-10 minutes to complete.

In [ ]:
predictor = training_model.deploy(initial_instance_count = 1, instance_type = 'ml.m5.xlarge')


## SageMaker FeatureStore During Inference
SageMaker FeatureStore can be useful in supplementing data for inference requests because of the low-latency GetRecord functionality. For this demo, we will be given a patientID and query our online FeatureGroup to build our inference request.

From the patient ID we left out in training set, we can choose one patient ID to test the real-time reference. In this example we choose patient `194`, but you can choose either one from the left out ID list for testing.

In [ ]:
id_for_test

To retrieve the data for a specific feature by its record identifier (patient ID we just randomly chose) from the online store, we can use the get_record function.

In [ ]:
patient_id = str(194)

# Helper to parse the feature value from the record.
def get_feature_value(record, feature_name):
    return str(list(filter(lambda r: r['FeatureName'] == feature_name, record))[0]['ValueAsString'])

clinical_response = featurestore_runtime.get_record(FeatureGroupName=clinical_feature_group_name, RecordIdentifierValueAsString=patient_id)
clinical_record = clinical_response['Record']
clinical_record

Then we choose the feature value from the retrieved feature list, exclude the record identifier ID, the event time, and the target variable, and build a list of values as the input to the predictor. 

In [ ]:
inference_request = [
    get_feature_value(clinical_record, 'age'),
    get_feature_value(clinical_record, 'anaemia'),
    get_feature_value(clinical_record, 'creatinine_phosphokinase'),
    get_feature_value(clinical_record, 'diabetes'),
    get_feature_value(clinical_record, 'ejection_fraction'),
    get_feature_value(clinical_record, 'high_blood_pressure'),
    get_feature_value(clinical_record, 'platelets'),
    get_feature_value(clinical_record, 'serum_creatinine'),
    get_feature_value(clinical_record, 'serum_sodium'),
    get_feature_value(clinical_record, 'sex'),
    get_feature_value(clinical_record, 'smoking'),
    get_feature_value(clinical_record, 'time')
]

The predictor will call our hosted model and give a prediction result. The model correctly predict the patient `194` is very likely (78% chance) to have a heart failure.

In [ ]:
import json

results = predictor.predict(','.join(inference_request), initial_args = {"ContentType": "text/csv"})
prediction = json.loads(results)
print (prediction)

## Clean Up Resources
You can delete the model endpoint and the FeatureGroup after you are done with this demo due to cost considerations.

In [ ]:
def cleanup():
    predictor.delete_endpoint()
    clinical_feature_group.delete()

# Further Read
* [SageMaker Feature Store Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html)
* [Store, Discover, and Share Machine Learning Features with Amazon SageMaker Feature Store](https://aws.amazon.com/blogs/aws/new-store-discover-and-share-machine-learning-features-with-amazon-sagemaker-feature-store/?sc_icampaign=launch_sagemaker-feature-store_reinvent20&sc_ichannel=ha&sc_icontent=awssm-6216&sc_iplace=ribbon&trk=ha_awssm-6216)  
* [Using streaming ingestion with Amazon SageMaker Feature Store to make ML-backed decisions in near-real time](https://aws.amazon.com/blogs/machine-learning/using-streaming-ingestion-with-amazon-sagemaker-feature-store-to-make-ml-backed-decisions-in-near-real-time/)
* [Fraud Detection using SageMaker FeatureStore](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-featurestore/sagemaker_featurestore_fraud_detection_python_sdk.ipynb)
